In [1]:
# https://discuss.pytorch.org/t/i-have-3-gpu-why-torch-cuda-device-count-only-return-1/7245/4
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

__Python VERSION: 3.8.3 (default, May 19 2020, 18:47:26) 
[GCC 7.3.0]
__pyTorch VERSION: 1.7.0
__CUDA VERSION
/usr/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 8003
__Number CUDA Devices: 4
__Devices
Active CUDA Device: GPU 0
Available devices  4
Current cuda device  0


In [2]:
%load_ext autoreload
%autoreload 2

import os
import json
import main
import random
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser

wandb: Currently logged in as: chucooleg (use `wandb login --relogin` to force relogin)


In [3]:
# arguments
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# args.data_path = '../Raw_Datasets/SET/4Attr-3Vals-2NestDepth-2CardsPerQ-Full-6561Train-6561Val.json'
args.data_path = '../Raw_Datasets/SimpleShatter/SimpleShatter-Full-1Train-10Val.json'
args.config_path = 'config.json'
args.generate_full_matrix = True
args.checkpoint_dir = 'checkpoints'
args.mode = 'train'
args.gpu = 1
args.approve_before_training = False

In [4]:
trainmodule, game_datamodule = main.main(args)

Global seed set to 42


---------data----------
num_attributes : 1
num_attr_vals : 10
num_cards_per_query : 10
nest_depth_int : 1
key_support_size : 11
query_support_size : 1024
max_len_q : 10
len_k : 1
hold_out : False
train_datapoints length : 5121
val_datapoints length : 5121
train_tokens length : 5121
val_tokens length : 5121
sparsity_estimate : 0.45463423295454547
vocab_size : 19
( : 11
) : 12
NULL : 13
SEP : 14
SOS : 15
EOS : 16
PAD : 17
PLH : 18
-----------------------
----------hparams----------
seed : 42
batch_size : 512
max_epochs : 200000
val_every_n_epoch : 200
d_model : 1024
embed_dropout : 0.0
model : generative
embedding_by_property : True
encoder : transformer
decoder : transformer
d_ff : 1024
num_heads : 2
N_enc : 1
N_dec : 1
attn_wt_tying_scheme : untie_QKVO
attn_wt_dropout : 0.0
heads_dropout : 0.0
pff_dropout : 0.0
representation_pos : 0
dotproduct_bottleneck : True
normalize_dotproduct : True
contrastive_use_infoNCE : True
loss_temperature_const : 1.0
loss_smoothing_const : 0.1
nonlinear_

/home/user/miniconda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory checkpoints/ContrastiveLearning-cardgame-Debug-Shattering/20210308-224819-SET;attr1-val10-nest1;generative;embedByProperty;d_model1024;dot-product;params6336.51K exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


   | Name                                                                 | Type                   | Params
-----------------------------------------------------------------------------------------------------------------
0  | metrics                                                              | ThresholdedMetrics     | 0     
1  | model                                                                | DecoderPredictor       | 6.3 M 
2  | model.inp_querykey_layer                                             | Sequential             | 34.8 K
3  | model.inp_querykey_layer.scaled_embed                                | ScaledEmbedding        | 19.5 K
4  | model.inp_querykey_layer.scaled_embed.embedding                      | Embedding              | 19.5 K
5  | model.inp_querykey_layer.position_encoder                            | LearnedPositionEncoder | 15.4 K
6  | model.inp_querykey_layer.embed_dropout                               | Dropout                | 0     
7  | model.querykey_de

/app/Contrastive-Learning-Benchmarking/SecondPass-CardGame-experiments/main.py:223: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():
wandb: wandb version 0.10.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



   | Name                                                                 | Type                   | Params
-----------------------------------------------------------------------------------------------------------------
0  | metrics                                                              | ThresholdedMetrics     | 0     
1  | model                                                                | DecoderPredictor       | 6.3 M 
2  | model.inp_querykey_layer                                             | Sequential             | 34.8 K
3  | model.inp_querykey_layer.scaled_embed                                | ScaledEmbedding        | 19.5 K
4  | model.inp_querykey_layer.scaled_embed.embedding                      | Embedding              | 19.5 K
5  | model.inp_querykey_layer.position_encoder                            | LearnedPositionEncoder | 15.4 K
6  | model.inp_querykey_layer.embed_dropout                               | Dropout                | 0     
7  | model.querykey_d

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/user/miniconda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/app/Contrastive-Learning-Benchmarking/SecondPass-CardGame-experiments/dataset.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt_binary_tensor = torch.tensor(gt_binary).long()


> /app/Contrastive-Learning-Benchmarking/SecondPass-CardGame-experiments/trainmodule.py(243)compute_topK_accuracy()
    241             topk_logit_poses = torch.topk(logits, k=query_tokens.shape[0])[1]
    242             import pdb; pdb.set_trace()
--> 243             acc = 0
    244             for pos in topk_logit_poses:
    245                 if pos in query_tokens:

ipdb> torch.topk(logits, k=query_tokens.shape[0])[1]
tensor([], device='cuda:1', dtype=torch.int64)
ipdb> query_tokens.shape[0]
0
ipdb> query_tokens
tensor([], device='cuda:1', dtype=torch.int64)
ipdb> X_querykey[b_i, 1:key_pos-1]
tensor([], device='cuda:1', dtype=torch.int64)
ipdb> X_querykey[b_i]
tensor([15, 10, 14, 10, 16, 17, 17, 17], device='cuda:1')
ipdb> key_poses[b_i]
tensor(2, device='cuda:1')
ipdb> key_pos
tensor(2, device='cuda:1')
ipdb> key_pos
tensor(2, device='cuda:1')
ipdb> querykey_logits[b_i][key_pos]
tensor([ 0.3801,  1.5944,  0.1427, -0.5441, -1.3192,  0.4459,  0.5777,  1.1486,
         0.9019,  0.

BdbQuit: 